## Run Once in Colab

In [ ]:
!git clone https://github.com/DrYogurt/Aero-Design-Team-Gamma.git
!pip install poetry==1.4.2

Cloning into 'Aero-Design-Team-Gamma'...
remote: Enumerating objects: 281, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 281 (delta 9), reused 15 (delta 6), pack-reused 254 (from 1)
Receiving objects: 100% (281/281), 41.62 MiB | 11.99 MiB/s, done.
Resolving deltas: 100% (123/123), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of poetry-plugin-export to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of virtualenv to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of virtualenv to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter co

In [ ]:
%cd Aero-Design-Team-Gamma
!poetry config virtualenvs.in-project true
!poetry install --no-ansi

/content/Aero-Design-Team-Gamma
Creating virtualenv aerospace-design-team-gamma in /content/Aero-Design-Team-Gamma/.venv
Installing dependencies from lock file
The lock file might not be compatible with the current version of Poetry.
Upgrade Poetry to ensure the lock file is read properly or, alternatively, regenerate the lock file with the `poetry lock` command.

Package operations: 137 installs, 1 update, 0 removals

  • Installing attrs (25.1.0)
  • Installing rpds-py (0.22.3)
  • Installing typing-extensions (4.12.2)
  • Installing referencing (0.36.2)
  • Installing six (1.17.0)
  • Installing jsonschema-specifications (2024.10.1)
  • Installing platformdirs (4.3.6)
  • Installing python-dateutil (2.9.0.post0)
  • Installing traitlets (5.14.3)
  • Installing types-python-dateutil (2.9.0.20241206)
  • Installing arrow (1.3.0)
  • Installing fastjsonschema (2.21.1)
  • Installing jsonschema (4.23.0)
  • Installing jupyter-core (5.7.2)
  • Installing pycparser (2.22)
  • Installing p

In [ ]:
VENV_PATH = "/content/Aero-Design-Team-Gamma/.venv/lib/python3.11/site-packages"
import os, sys
LOCAL_VENV_PATH = '/content/venv' # local notebook
os.symlink(VENV_PATH, LOCAL_VENV_PATH) # connect to directory in drive
sys.path.insert(0, LOCAL_VENV_PATH)

# Code Begins Here
## Imports and Variables
We begin by importing our variables and setting up the system of equations

In [5]:
from ambiance import Atmosphere
import numpy as np

from global_variables.solver import EquationSystem
from global_variables.registry import VariableRegistry, Variable

registry = VariableRegistry("aero_vars.yaml")

In [6]:
constraint_vars = {"R","S_TO","sigma_max"}
constraints_system = EquationSystem(registry,constraint_vars)
constraints_solver = constraints_system.create_solver()
optimizer_vars = {"W_max","C_L_full"}
optimizer_system = EquationSystem(registry,optimizer_vars)
optimizer_eqs = optimizer_system.create_solver()

/home/dryogurt/Documents/Habitats/Aero-Design-Team-Gamma/global_variables/solver.py:125: UserWarning: No solution found for target W_max, treating as input
  warnings.warn(f"No solution found for target {target}, treating as input")


In [7]:
print(f"Constraints: {constraints_solver}")
print(f"Optimizations: {optimizer_eqs}")
print(f"All inputs: {constraints_system.inputs}")

Constraints: {'R': <function _lambdifygenerated at 0x754367583100>, 'sigma_max': <function _lambdifygenerated at 0x7543675de020>, 'S_TO': <function _lambdifygenerated at 0x7543675ddf80>}
Optimizations: {'W_max': <function EquationSystem.create_solver.<locals>.<lambda> at 0x7543674a6ac0>, 'C_L_full': <function _lambdifygenerated at 0x7543675de5c0>}
All inputs: {'S_', 'mu', 'T_A0', 'rho_h', 'W_max', 'V', 'W_pax', 'rho', 'b', 'C_Lmax', 'e', 'n_pax', 'We_Wmax', 'C_D0', 'TSFC'}


In [8]:
def velocity(M,h):
    try:
        atmo = Atmosphere(np.clip(h * 0.3048,a_min=-5004,a_max=81020))
        return atmo.speed_of_sound * M / 0.3048
    except:
        raise ValueError(f"altitude {h} is out of bounds!")

def rho_func(h):
    try:
        atmo = Atmosphere(np.clip(h * 0.3048,a_min=-5004,a_max=81020))
        return atmo.density * 0.00194032
    except:
        raise ValueError(f"altitude {h} is out of bounds!")


## The Cost function
here, I'm converting from human legible parameters to the correct units for the optimizer, and setting up a function which takes in all of our inputs and produces a single scalar result, which is 0 at the optimal plane design and higher otherwise.

In [27]:
target_sigma_max = rho_func(4.3e4) / rho_func(0)
target_R = 1.02e4 * 6076.12
target_S_TO = .99e5

def constraint_cost(x):
    return np.where(x > 0, np.exp(x)-1, -x)

def convert_param_to_dict(x):
    x = np.atleast_2d(x)
    return {
        'T_A0':x[:,0],
        'TSFC':x[:,1]/3600,
        'e':x[:,2],
        'rho':rho_func(0), # fixed, density at sea level
        'rho_h':rho_func(x[:,7]), # fixed, density at sea level
        'W_max':x[:,3],
        'n_pax':1255, # fixed, num of passengers
        'S_'
         :x[:,4],
        'C_Lmax':x[:,5],
        'W_pax':205, # fixed, 205
        'V':velocity(x[:,6],x[:,7]), #0.8 < M < 1, cruising altitude
        'C_D0':x[:,8],
        'We_Wmax':x[:,9],
        'mu':0.02, # fixed, dry runway
        'b':x[:,10]
    }

In [35]:
def cost_function(x, debug=False):
    all_inputs = convert_param_to_dict(x)
    sigma_max_raw = constraints_solver['sigma_max'](**all_inputs)
    R_raw = constraints_solver['R'](**all_inputs)
    S_TO_raw = constraints_solver['S_TO'](**all_inputs)

    sigma_max_diff = (sigma_max_raw - target_sigma_max) / target_sigma_max
    R_diff = (target_R - R_raw) / target_R
    S_TO_diff = (S_TO_raw - target_S_TO) / target_S_TO

    constraint_costs = {
        'sigma_max_cost': constraint_cost(sigma_max_diff),
        'R_cost': constraint_cost(R_diff),
        'S_TO_cost': constraint_cost(S_TO_diff)
    }
    total_constraint_cost = sum(constraint_costs.values())

    opt_inputs = {inp: all_inputs[inp] for inp in optimizer_system.inputs}
    C_L_full = optimizer_eqs["C_L_full"](**opt_inputs)
    W_max = all_inputs["W_max"]
    C_L_cost = (C_L_full - 0.46) / 0.46 # weighted cost for C_L
    W_cost = (W_max - 2e6) / 2e6

    weighted_costs = {
        'W_cost': W_cost,
        'C_L_cost': C_L_cost**2
    }
    if np.any(np.array(list(constraint_costs.values())) > 0):
        total_cost = total_constraint_cost + C_L_cost**2
    else:
        total_cost = total_constraint_cost + sum(weighted_costs.values())

    if debug:
        print("Cost Breakdown:")
        for key, value in constraint_costs.items():
            print(f"  {key}: {value:.6f}")
        for key, value in weighted_costs.items():
            print(f"  {key}: {value:.6f}")
        print(f"  Total Cost: {total_cost:.6f}")

    return total_cost


## Bounds: Change this to change how the optimizer will perform
Below is a list of bounds, if you want to tweak how the optimizer will perform, mess around with them!

In [36]:
from scipy.optimize import minimize
import pyswarms as ps

# Define bounds for each parameter
bounds = [
    (3.5e5, 5.5e5),     # T_A0planes
    (0.4, 0.43),     # TSFC (converted to per hour in cost function)
    (0.9, 0.95),    # e
    (0, 1.95e6),     # W_max
    (1e4, 2e4),  # S_
    (1.5, 1.8),     # C_Lmax
    (0.8, 0.85),    # Mach number
    (3e4, 4.0e4),     # Cruise Altitude
    (0.02, 0.022), # C_D0
    (0.45, 0.55),   # We_Wmax
    (250, 345)      # b
]

### The Optimizer functions themselves

In [37]:

def scipy_optimizer(cost_function, x0=None):
    """
    Optimize using SciPy's L-BFGS-B algorithm

    Args:
        cost_function: Function to minimize
        x0: Initial guess (optional)

    Returns:
        tuple: (optimal parameters, optimal cost)
    """
    if x0 is None:
        x0 = [5e5, 0.45, 0.92, 2e6, 10200, 1.5, 0.85, 3.5e4, 0.02, 0.48, 315]

    result = minimize(
        cost_function,
        x0,
        method='L-BFGS-B',
        bounds=bounds,
        options={
            'maxiter': 1000,
            'ftol': 1e-8,
            'disp': True
        }
    )

    return result.x, result.fun
def pyswarms_optimizer(cost_function, n_particles=50, iters=1000,center=None):
    """
    Optimize using PySwarms' global best PSO

    Args:
        cost_function: Function to minimize
        n_particles: Number of particles in swarm
        iters: Number of iterations

    Returns:
        tuple: (optimal parameters, optimal cost)
    """
    # Convert bounds to numpy arrays for PySwarms
    lb = np.array([b[0] for b in bounds])
    ub = np.array([b[1] for b in bounds])

    # Initialize swarm
    options = {
        'c1': 0.5,    # cognitive parameter
        'c2': 0.3,    # social parameter
        'w': 0.9,     # inertia weight
        'k': 3,       # number of neighbors to look at
        'p': 2        # minkowski p-norm (2 = euclidean)
    }

    # Create optimizer object
    optimizer = ps.single.LocalBestPSO(
        n_particles=n_particles,
        dimensions=len(bounds),
        options=options,
        bounds=(lb, ub),
        center=[5e5, 0.45, 0.9, 2e6, 10200, 1.5, 0.85, 3.5e4, 0.02, 0.48, 315] if center is None else center
    )

    # Optimize
    best_cost, best_pos = optimizer.optimize(
        cost_function,
        iters=iters,
        verbose=True
    )

    return best_pos, best_cost

In [38]:
def print_output(x):
    b = x[10]
    S = x[4]
    M = x[6]
    h = x[7]
    dict_inputs = convert_param_to_dict(x)
    #print(dict_inputs)
    sigma_max_raw = constraints_solver['sigma_max'](**dict_inputs)
    R_raw = constraints_solver['R'](**dict_inputs)
    S_TO_raw = constraints_solver['S_TO'](**dict_inputs)
    max_h = Atmosphere.from_density(Atmosphere(0).density*sigma_max_raw).h / .3048
    opt_inputs = {inp: dict_inputs[inp] for inp in optimizer_system.inputs}
    C_L_full = optimizer_eqs["C_L_full"](**opt_inputs)
    print(f"""
    wingspan of {b:.2f} ft; planform area of {S:.1f} ft^2; AR of {b**2 / S:.3f}.
    oswald efficiency of {x[2]:.2f}.

    cruise at M {M:.3f} at  {h:.1f} ft; maximum altitude of {max_h[-1]} ft
    range of {R_raw[-1]/6076.11:.1f} nm; TSFC of {x[1]:.3f} /hr; parasitic drag below {x[8]:.4f}

    takeoff distance of {S_TO_raw[-1]:.1f} ft; C_L max of {x[5]:.3f};
    gross takeoff weight of {x[3]:.1f} lbs; T_A0 of {x[0]:.3f} lbs

    C_L at takeoff {C_L_full[-1]:.3f}; empty weight fraction of {x[9]:.3f}
    """)

### Running the optimizers

In [39]:
print("Optimizing with SciPy L-BFGS-B...")
best_params_scipy, best_cost_scipy = scipy_optimizer(cost_function)
print(f"Best parameters (SciPy): {best_params_scipy}")
print(f"Best cost (SciPy): {best_cost_scipy}")

print("\nOptimizing with PySwarms PSO...")
best_params_pso, best_cost_pso = pyswarms_optimizer(cost_function,
                                                    n_particles=20000,
                                                    iters=2000,
                                                    center = best_params_scipy)
print(f"Best parameters (PSO): {best_params_pso}")
print(f"Best cost (PSO): {best_cost_pso}")

print("--Scipy--")
print_output(best_params_scipy)
print("--Particle Swarm--")
print_output(best_params_pso)

Optimizing with SciPy L-BFGS-B...


2025-01-31 15:37:55,083 - pyswarms.single.local_best - INFO - Optimize for 2000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 3, 'p': 2}


Best parameters (SciPy): [5.00000000e+05 4.19367786e-01 9.12886745e-01 1.95000000e+06
 1.02000001e+04 1.50000000e+00 8.50000000e-01 3.50000000e+04
 2.10047580e-02 4.69369058e-01 3.14999768e+02]
Best cost (SciPy): 1.312514329716811

Optimizing with PySwarms PSO...


pyswarms.single.local_best: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|2000/2000, best_cost=0.972
2025-01-31 15:39:27,536 - pyswarms.single.local_best - INFO - Optimization finished | best cost: 0.9724703707175554, best pos: [4.11317802e+05 4.08691057e-01 9.21254386e-01 1.73702706e+06
 1.04281644e+04 1.63226387e+00 8.47758315e-01 3.12769306e+04
 2.04655631e-02 4.50403787e-01 3.40219374e+02]


Best parameters (PSO): [4.11317802e+05 4.08691057e-01 9.21254386e-01 1.73702706e+06
 1.04281644e+04 1.63226387e+00 8.47758315e-01 3.12769306e+04
 2.04655631e-02 4.50403787e-01 3.40219374e+02]
Best cost (PSO): 0.9724703707175554
--Scipy--

    wingspan of 315.00 ft; planform area of 10200.0 ft^2; AR of 9.728.
    oswald efficiency of 0.91.

    cruise at M 0.850 at  35000.0 ft; maximum altitude of 43000.00068421968 ft
    range of 10200.0 nm; TSFC of 0.419 /hr; parasitic drag below 0.0210

    takeoff distance of 10330.9 ft; C_L max of 1.500;
    gross takeoff weight of 1950000.0 lbs; T_A0 of 500000.000 lbs

    C_L at takeoff 0.757; empty weight fraction of 0.469
    
--Particle Swarm--

    wingspan of 340.22 ft; planform area of 10428.2 ft^2; AR of 11.100.
    oswald efficiency of 0.92.

    cruise at M 0.848 at  31276.9 ft; maximum altitude of 43082.20354050603 ft
    range of 10050.3 nm; TSFC of 0.409 /hr; parasitic drag below 0.0205

    takeoff distance of 9014.8 ft; C_L max of 1